In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Необходимые библиотеки 

In [ ]:
# for Crawler
!pip3 install requests
!pip3 install bs4
# lemmatizing, tokenization, MA for hindi
!pip3 install stanza
# lemmatizing, tokenization, MA for russian
!pip3 install pymystem3
!pip3 install nltk
!pip3 install pymorphy2[fast]
!pip3 install -U pymorphy2-dicts-ru
import nltk
nltk.download('punkt')

## Краулер для сбора стихов Пушкина

In [ ]:
import requests
from bs4 import BeautifulSoup


class Crawler:
    def __init__(self, base_url=None):
        self.poems = []
        self.base_url = base_url
    
    def is_poem(self, tag):
        return tag.has_attr('class') and tag['class'][0] == 'poem'

    def collect_poems(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        soup = soup.find(id='mw-content-text')
        poem = soup.find(self.is_poem)
        if poem is not None:
            self.poems.append(poem.text)
        else:
            for tag in soup.find_all('li'):
                if not tag.has_attr('class'):
                    a_tag = tag.find('a')
                    if a_tag is not None:
                        if not a_tag.has_attr('class'):
                            self.collect_poems(self.base_url + a_tag.get('href'))

In [ ]:
base_url = 'http://kavitakosh.org'
crawler = Crawler(base_url)
url = 'http://kavitakosh.org/kk/%E0%A4%85%E0%A4%B2%E0%A5%87%E0%A4%95%E0%A5%8D%E0%A4%B8%E0%A4%BE%E0%A4%A8%E0%A5%8D%E0%A4%A6%E0%A4%B0_%E0%A4%AA%E0%A5%82%E0%A4%B6%E0%A5%8D%E0%A4%95%E0%A4%BF%E0%A4%A8#.E0.A4.95.E0.A4.B5.E0.A4.BF.E0.A4.A4.E0.A4.BE-.E0.A4.B8.E0.A4.82.E0.A4.97.E0.A5.8D.E0.A4.B0.E0.A4.B9'
crawler.collect_poems(url)

with open('/content/gdrive/MyDrive/kursuch/data/pushkin_poems.txt', 'w') as f:
    for poem in crawler.poems:
        f.write(poem)
        f.write('-' * 10)

## Обработка XML-а

### Транслитератор 1 - Pandey

In [ ]:
  # letters that hindi does not use
  # 'ऎ' : 'e', 'ऒ' : 'o', 'ऍ' : 'ê', 'ऑ' : 'ô', 'ॅ' : 'ê', 'ॉ' : 'ô', 'ॻ' : 'g', 'ॼ' : 'j', 'ॸ' : 'ḍ', 'ॾ' : 'd', 'ॿ' : 'b', 'ऋ' : 'r̥', 'ॠ' : ' r̥̄', 'ऌ' : 'l̥', 'ॡ' : ' l̥̄'
        

In [ ]:
import re

class PandeyTransliterator:
    def __init__(self):
        self.conversiontable = { 'ॐ' : 'oṁ', 'ऀ' : 'ṁ', 'ँ' : 'ṃ', 'ं' : 'ṃ', 'ः' : 'ḥ', 'ऽ' : '\'', '्' : '', '़' : '', '०' : '0', '१' : '1', '२' : '2', '३' : '3', '४' : '4', '५' : '5', '६' : '6', '७' : '7', '८' : '8', '९' : '9', 
                                'ꣳ' : 'ṁ', '।' : '.', '॥' : '..', ' ' : ' ',
                                'अ' : 'a', 'आ' : 'ā', 'ा' : 'ā',
                                'इ' : 'i', 'ई' : 'ī', 'ि' : 'i', 'ी' : 'ī',
                                'उ' : 'u', 'ऊ' : 'ū', 'ु' : 'u', 'ू' : 'ū',
                                'ए' : 'e', 'े' : 'e', 'ओ' : 'o', 'ो' : 'o',
                                'ऍ' : 'e', 'ॅ' : 'e', 'ऑ' : 'o', 'ॉ' : 'o',
                                'ऐ' : 'ai',  'ै' : 'ai', 'औ' : 'au', 'ौ' : 'au',
                                'क' : 'k', 'ख' : 'kh', 'ग' : 'g', 'घ' : 'gh', 'ङ' : 'ṅ',
                                'च' : 'c', 'छ' : 'ch', 'ज' : 'j', 'झ' : 'jh', 'ञ' : 'ñ',
                                'ट' : 'ṭ', 'ठ' : 'ṭh',  'ढ' : 'ḍh', 'ड' : 'ḍ', 'ड़' : 'ṛ', 'ढ़' : 'ṛh', 'ण' : 'ṇ', 
                                'त' : 't', 'थ' : 'th', 'द' : 'd', 'ध' : 'dh', 'न' : 'n',
                                'प' : 'p', 'फ' : 'ph', 'ब' : 'b', 'भ' : 'bh', 'म' : 'm', 
                                'य' : 'y', 'र' : 'r', 'ल' : 'l', 'ळ' : 'ḷ',
                                'ह' : 'h', 'व' : 'v', 'स' : 's', 'श' : 'ś', 'ष' : 'ṣ', 
                                'क़' : 'q', 'ख़' : 'x', 'ग़' : 'ġ', 'ज़' : 'z', 'फ़' : 'f', 
                                }
        self.consonants = '\u0915-\u0939\u0958-\u095F\u0978-\u097C\u097E-\u097F' 
        self.vowelsigns = '\u093E-\u094C\u093A-\u093B\u094E-\u094F\u0955-\u0957' 
        self.nukta = '\u093C' 
        self.virama = '\u094D' 
        self.devanagarichars = '\u0900-\u097F\u1CD0-\u1CFF\uA8E0-\uA8FF'

    def __deva_to_latin__(self, text):
        word = text.strip() 
        curr = ''  
        for index, char in enumerate(word):   
            if re.match('[' + self.devanagarichars + ']', char): 
                if re.match('[' + self.consonants + ']', char):
                    nextchar = word[(index + 1) % len(word)]
                    if nextchar:
                        if re.match('[' + self.nukta + ']', nextchar): 
                            cons = char + nextchar 
                            nukta_present = 1 
                        else: 
                            cons = char 
                            nukta_present = 0 
                            if nukta_present: 
                                nextchar = word[(index + 2) % len(word)]   
                
                        if re.match('[' + self.vowelsigns + self.virama +']', nextchar): 
                            trans = self.conversiontable.get(cons, '') 
                            curr = curr + trans 
                        else: 
                            trans = self.conversiontable.get(cons, '') 
                            trans = trans + "a" 
                            curr = curr + trans # transliterate all other chars 
                else: 
                    trans = self.conversiontable.get(char, '') 
                    curr = curr + trans # char is not Devanagari. output char to curr 
            else: 
                curr = curr + char 
        return curr

    def transform(self, text):
        word_syllables = [] 
        all_words = [] 
        for word in text.split(): 
            latin_output = self.__deva_to_latin__(word) 
            all_words.append(latin_output) 
            joined_all_words = ' '.join(all_words) 
        return joined_all_words


### Транслитератор 2: indic-trans


#### indic-trans устроен сложнее, чем Pandey, он поддерживает 16 индийских языков, включая английский, поэтому устанавливаем только нужный нам сетап

In [ ]:
!git clone https://github.com/libindic/indic-trans.git
!cd indic-trans; pip3 install -r requirements.txt

In [ ]:
!mv -v /content/indic-trans/* /content/

In [ ]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x7f7674a85150>)

In [ ]:
from indictrans import Transliterator

trn = Transliterator(source='hin', target='eng', build_lookup=True)
hin = 'आज मौसम अच्छा है। इसलिए हम आज खेल सकते हैं!'
eng = trn.transform(hin)
print(eng)

/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/indictrans/_utils/ctranxn.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/indictrans/_utils/sparseadd.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/indictrans/_decode/viterbi.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2).

aaj mausam acha he. isliye ham aaj khel sakate hai!


### Теперь, имея все нужные библиотеки для обработки, можем собрать XML

In [ ]:
import xml.etree.ElementTree as ET
import stanza


class XmlProcessor:
    def __init__(self, setup=False, transcr='indic-trans'):
        if setup:
            stanza.download('hi')
        if transcr == 'indic-trans':
            self.transliterator = Transliterator(source='hin', target='eng', build_lookup=True)
        elif transcr == 'pandey':
            self.transliterator = Transliterator()
        else:
            raise ValueError('XmlProcessor supports only "indic-trans" and "pandey" transliterators at the moment')
        self.pipeline = stanza.Pipeline(lang='hi')
        # соответствия тегов UD и НКРЯ
        self.pos_from_ud_to_gr_table = {
            "ADJ": "A",
            "ADP": "PR",
            "ADV": "ADV",
            "AUX": "AUX",
            "CCONJ": "CONJ",
            "DET": "DET",
            "INTJ": "INTJ",
            "NOUN": "S",
            "NUM": "NUM",
            "PART": "PART",
            "PRON": "PRO",
            "PROPN": "persn",
            "SCONJ": "CONJ",
            "VERB": "V",
            "X": "X",
            "PUNCT": "PUNCT"
        }
        self.feats_from_ud_to_gr_table = {
            "Gender": {
                "Fem":"f",
                "Masc":"m"
            },
            "AdpType": {
                "POST":"POSL",
            },
            "Number": {
                "Plur":"pl",
                "Sing":"sg"
            },
            "Case": {
                "Acc":"acc", 
                "Dat":"dat", 
                "Erg":"erg", 
                "Gen":"gen", 
                "Ine":"voc", 
                "Ins":"ins", 
                "Nom":"nom"
            },
            "VerbForm": {
                "Conv":"ger", 
                "Fin":"fin", 
                "Inf":"inf", 
                "Part":"partcp"
            },
            "Mood": {
                "Imp":"imper", 
                "Ind":"indic", 
                "Sub":"sub"
            },
            "Tense": {
                "Fut":"fut", 
                "Past":"praet", 
                "Pres":"praes"
            },
            "Voice": {
                "Act":"act", 
                "Pass":"pass"
            },
            "Person": {
                "1":"1p", 
                "2":"2p", 
                "3":"3p"
            }
    }

    def __parse_ud_feats__(self, token):   # разбираем выдачу UD
        dummy = dict()
        ud = [token.upos, dummy]
        if token.feats is None:
            return ud
        feats = dict()
        raw_feats = token.feats.split('|')
        for raw_feat in raw_feats:
            split = raw_feat.split('=')
            feats[split[0]] = split[1]
        ud[1] = feats
        return ud

    def __from_ud_to_gr__(self, ud_feats):    # преобразуем теги UD в формат НКРЯ
        gr_feats = []
        if len(ud_feats) == 0:
            return ""
        if ud_feats[0] in self.pos_from_ud_to_gr_table:
            gr_feats.append(self.pos_from_ud_to_gr_table[ud_feats[0]])
        
        for key, value in ud_feats[1].items():
            if key in self.feats_from_ud_to_gr_table:
                if value in self.feats_from_ud_to_gr_table[key]:
                    gr_feats.append(self.feats_from_ud_to_gr_table[key][value])
        return ','.join(gr_feats)


    def __process__(self, root):                             # дополняем исходный XML леммой, транслитерацией (по выбору пользователя одной из двух представленных) 
        for para in root[1].findall('para'):                 # леммой, грамматическими категориями формата UD и НКРЯ
            Allses = para.findall('se')
            for se in Allses:
                if se.attrib['variant_id'] == "1":
                    phrase = para[1].text
                    if phrase is None:
                        continue
                    else:
                        para[1].text = '\n'
                        doc = self.pipeline(phrase)
                        for sent in doc.sentences:
                            for token in sent.words:
                                ud_feats = self.__parse_ud_feats__(token)
                                if len(ud_feats) == 1:
                                    ud = ud_feats[0]
                                else:
                                    ud = [ud_feats[0]]
                                    ud.extend(list(ud_feats[1].values()))
                                    ud = ','.join(ud)
                                w_tag = ET.SubElement(se, 'w')
                                w_tag.text = token.text
                                ET.SubElement(
                                    w_tag,
                                    'ana',
                                    attrib={
                                        "transcr": self.transliterator.transform(token.text),
                                        "lex": token.lemma,
                                        "transcr_lex": self.transliterator.transform(token.lemma),
                                        "ud": ud,
                                        "gr": self.__from_ud_to_gr__(ud_feats)
                                    }
                                )


    def __remove_odd_tags__(self, root):
        for para in root[1].findall('para'):
            weight = para.find('weight')
            para.remove(weight)
            se = para.findall('se')
            for i in se:
                i.attrib.pop('variant_id', None)    

    def process(self, input_path, output_path):
        tree = ET.parse(input_path)
        root = tree.getroot()
        self.__process__(root)
        self.__remove_odd_tags__(root)
        tree.write(output_path)

In [ ]:
xml_processor = XmlProcessor()

2022-05-24 22:55:34 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |
| lemma     | hdtb    |
| depparse  | hdtb    |

2022-05-24 22:55:34 INFO: Use device: cpu
2022-05-24 22:55:34 INFO: Loading: tokenize
2022-05-24 22:55:34 INFO: Loading: pos
2022-05-24 22:55:34 INFO: Loading: lemma
2022-05-24 22:55:34 INFO: Loading: depparse
2022-05-24 22:55:35 INFO: Done loading processors!


In [ ]:
path_to_xml = '/content/gdrive/MyDrive/kursuch/data/tolstoy-kavkazski_plennik-aligned.xml'
output_path = path_to_xml[:-4] + '_PROCESSED.xml'
xml_processor.process(path_to_xml, output_path)

In [ ]:
path_to_xml = '/content/gdrive/MyDrive/kursuch/data/chehov-kashtanka-aligned.xml'
output_path = path_to_xml[:-4] + '_PROCESSED.xml'
xml_processor.process(path_to_xml, output_path)

## преобразование файлов XML в JSON [для выгрузки на Цакорпус]

### Sample json



формат нужного json-а
https://tsakorpus.readthedocs.io/en/latest/data_model.html

# summary того, как должен выглядеть json

    json_string = {
        "meta": {
            "author": "Tolstoy",
            "title": "Kavkazski plennik",
            "year": 1872
        },
        "sentences": [
        {
            "text": "काकेशस का कैदी",
            "words": [
                {
                    "wf": "काकेशस",
                    "wtype": "word",
                    "off_start": 0,
                    "off_end": 4,
                    "next_word": 1,
                    "sentence_index": 0,
                    "sentence_index_neg": -3,
                    "ana": [
                        {
                            "lex": "काकेशस",
                            "gr.pos": "PROPN",
                            "gr.feats": "Acc,Masc,Sing,3"
                        }
                    ]
                },
                {
                    "wf": "का",
                    "wtype": "word",
                    "off_start": 6,
                    "off_end": 7,
                    "next_word": 2,
                    "sentence_index": 1,
                    "sentence_index_neg": -2,
                    "ana": [
                        {
                            "lex": "का",
                            "gr.pos": "ADP",
                            "gr.feats": "Post,Nom,Masc,Sing"
                        }
                    ]
                },
                {
                    "wf": "कैदी",
                    "wtype": "word",
                    "off_start": 8,
                    "off_end": 10,
                    "next_word": 3,
                    "sentence_index": 2,
                    "sentence_index_neg": -1,
                    "ana": [
                        {
                            "lex": "कैदी",
                            "gr.pos": "NOUN",
                            "gr.feats": "Nom,Masc,Sing,3"
                        }
                    ]
                }
            ],
            "lang": 1,
            "para_alignment": [
                {
                    "off_start": 0,
                    "off_end": 10,
                    "para_id": 0
                }
            ]
        },


### преобразуем в JSON

In [ ]:
from nltk.inference.tableau import Agenda
import string
import json
import xml.etree.ElementTree as ET
import pymorphy2
from nltk import word_tokenize


class XmlToJson:
    def __init__(self, setup_hindi=False, transcr='indic-trans'):
        self.morph = pymorphy2.MorphAnalyzer()
        if setup_hindi:
            stanza.download('hi')
        if transcr == 'indic-trans':
            self.transliterator = Transliterator(source='hin', target='eng', build_lookup=True)
        elif transcr == 'pandey':
            self.transliterator = Transliterator()
        else:
            raise ValueError('Supports only "indic-trans" and "pandey" transliterators at the moment')
        self.pipeline = stanza.Pipeline(lang='hi')

    def __is_punct__(self, token):
        return token in string.punctuation
    
    def __get_sentences_from_xml__(self, root):
        sentences = []
        for para in root[1].findall('para'):
            ru, hindi = para.findall('se')
            sentences.append((ru.text, hindi.text))
        return sentences

    def __create_sentence__(self, text, words, lang, para_id):
        return {
            'text': text,
            'words': words,
            'lang': lang,
            'para_alignment': [
                {
                    'off_start': 0,
                    'off_end': len(text) - 1,
                    'para_id': para_id
                }
            ]
        }
    
    def __create_word__(self, wf, wtype, off_start, off_end, next_word, sentence_index, 
                        sentence_index_neg, ana):
        word = {
                'wf': wf,
                'wtype': wtype,
                'off_start': off_start,
                'off_end': off_end,
                'next_word': next_word,
            }
        if sentence_index is not None:
            word['sentence_index'] = sentence_index
            word['sentence_index_neg'] = sentence_index_neg
            word['ana'] = ana
        return word
    
    def __get_ru_sentence_len__(self, tokens):
        ans = len(tokens)
        if 'PNCT' in self.morph.parse(tokens[0])[0].tag:
            ans -=1
        if 'PNCT' in self.morph.parse(tokens[-1])[0].tag:
            ans -= 1
        return ans

    def __get_hindi_sentence_len__(self, doc):
        ans = len(doc.sentences[0].words)
        if doc.sentences[0].words[0].upos == 'PUNCT':
            ans -= 1
        if doc.sentences[0].words[-1].upos == 'PUNCT':
            ans -= 1
        return ans

    def __make_russian_sentence__(self, ru, para_id):
        tokens = word_tokenize(ru)
        words = []
        off_start, off_end = 0, 0
        curr_sentence_index = 0
        size = self.__get_ru_sentence_len__(tokens)
        for i, token in enumerate(tokens):
            off_start = off_end
            off_end = off_start + len(token)
            sentence_index = curr_sentence_index
            sentence_index_neg = sentence_index - size
            curr_sentence_index += 1
            wtype = 'punct'
            ana = None
            analysis = self.morph.parse(token)[0]
            tag = analysis.tag
            if (i == 0 or i + 1 == len(tokens)) and 'PNCT' in tag:
                sentence_index, sentence_index_neg = None, None
                curr_sentence_index -= 1
            if 'PNCT' not in tag:
                wtype = 'word'
                ana = [
                    {
                        'lex': analysis.normal_form,
                        'gr.pos': tag.POS
                    }
                ]
                feats = str(tag).split(',')
                if len(feats) != 1:
                    ana[0]['gr.feats'] = ','.join(feats[1:])
            words.append(self.__create_word__(
                    token, wtype, off_start, off_end, i + 1, sentence_index, sentence_index_neg, ana
                )
            )
        return self.__create_sentence__(ru, words, 0, para_id)

    
    def __make_hindi_sentence__(self, hindi, para_id):
        doc = self.pipeline(hindi)
        words = []
        off_start, off_end = 0, 0
        curr_sentence_index = 0
        index = 0
        size = self.__get_hindi_sentence_len__(doc)
        for sent in doc.sentences:
            for token in sent.words:
                off_start = off_end
                off_end = off_start + len(token.text)
                sentence_index = curr_sentence_index
                sentence_index_neg = sentence_index - size
                curr_sentence_index += 1
                ana = None
                wtype = 'punct'
                if (index == 0 or index + 1 == len(doc.sentences[0].words)) and token.upos == 'PUNCT':
                    sentence_index, sentence_index_neg = None, None
                    curr_sentence_index -= 1
                if token.upos != 'PUNCT':
                    wtype = 'word'
                    ana = [
                        {
                            'lex': token.lemma,
                            'transcr': self.transliterator.transform(token.text),
                            'transcr.lex': self.transliterator.transform(token.lemma),
                            'gr.pos': token.upos,
                        }
                    ]
                    if token.feats is not None:
                        ana[0]['gr.feats'] = token.feats

                words.append(self.__create_word__(
                        token.text, wtype, off_start, off_end, index + 1, sentence_index, sentence_index_neg, ana
                    )
                )
                index += 1
        return self.__create_sentence__(hindi, words, 1, para_id)

    def convert(self, path_to_xml, output_path, meta={}):
        tree = ET.parse(path_to_xml)
        root = tree.getroot()
        sentences = self.__get_sentences_from_xml__(root)
        json_dict = {
            'meta': meta,
            'sentences': []
        }
        para_id = 0
        for ru, hindi in sentences:
            if ru is None or hindi is None:
                continue
            json_dict['sentences'].append(self.__make_russian_sentence__(ru, para_id))
            json_dict['sentences'].append(self.__make_hindi_sentence__(hindi, para_id))
            para_id += 1
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(json_dict, f, indent=4)

In [ ]:
json_converter = XmlToJson()

2022-05-24 23:09:35 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |
| lemma     | hdtb    |
| depparse  | hdtb    |

2022-05-24 23:09:35 INFO: Use device: cpu
2022-05-24 23:09:35 INFO: Loading: tokenize
2022-05-24 23:09:35 INFO: Loading: pos
2022-05-24 23:09:35 INFO: Loading: lemma
2022-05-24 23:09:35 INFO: Loading: depparse
2022-05-24 23:09:35 INFO: Done loading processors!


In [ ]:
path_to_xml = '/content/gdrive/MyDrive/kursuch/data/tolstoy-kavkazski_plennik-aligned.xml'
output_path = path_to_xml[:-4] + '_PROCESSED_JSON1.json'
meta = {
    'author': 'Tolstoy',
    'name': 'Kavkazski plennik'
}
json_converter.convert(path_to_xml, output_path, meta)

In [ ]:
path_to_xml = '/content/gdrive/MyDrive/kursuch/data/chehov-kashtanka-aligned.xml'
output_path = path_to_xml[:-4] + '_PROCESSED_JSON1.json'
meta = {
    'author': 'Chehov',
    'name': 'Kashtanka'
}
json_converter.convert(path_to_xml, output_path, meta)